In [9]:
!pip install better-profanity

In [10]:
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import col
from better_profanity import profanity
from operator import add

# spark_session = SparkSession\
#         .builder\
#         .master("spark://sparkmaster:7077") \
#         .appName("test")\
#         .config("spark.dynamicAllocation.enabled", True)\
#         .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
#         .config("spark.shuffle.service.enabled", False)\
#         .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
#         .config("spark.executor.cores",2)\
#         .config("spark.driver.port",9999)\
#         .config("spark.blockManager.port",10005)\
#         .getOrCreate()
# spark_context = spark_session.sparkContext

spark = SparkSession.builder \
    .appName("MongoDB Integration") \
    .master("spark://sparkmaster:7077") \
    .config("spark.mongodb.read.connection.uri", "mongodb://mongo_db:27017/reddit.reddit_data?readPreference=primaryPreferred") \
    .config("spark.mongodb.input.uri", "mongodb://mongo_db:27017/reddit.reddit_data") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.0') \
    .config("spark.dynamicAllocation.enabled", True)\
    .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
    .config("spark.shuffle.service.enabled", False)\
    .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
    .getOrCreate()

spark_session = spark.sparkContext

spark_session.setLogLevel("INFO")



In [11]:
# Load data from MongoDB
df = spark.read.format("mongo").option('limit', 160000).load()
#df.printSchema()
reddit_rdd = df.rdd

profanity.load_censor_words()

# reddit_rdd.count()

In [12]:
reddit_tuple = df.select(col("subreddit"), col("normalizedBody")).rdd
subset_tuple = reddit_tuple.map(lambda x: (x[0], x[1]))

In [13]:

censored_tupple = subset_tuple.map(lambda x: (x[0],profanity.censor(x[1])))
count_of_stars = censored_tupple.map(lambda x: (x[0], x[1].count("****")))

In [14]:
# count_of_stars_tuple = count_of_stars.map(lambda x: (x[0], x[2]))

# reduced_count = count_of_stars.reduceByKey(add)
# .sortBy(lambda x: x[1], ascending=False)

In [15]:

censored_tupple.take(5)

[('funny',
  'Art is about the hardest thing to categorize in terms of good and bad. To consider one work or artist as dominate over another comes down to personal opinion. Sure some things maybe blatantly better than other works, but it ultimately lies with the individual. I personally enjoy the work of "street artists" (using quotations not to be sarcastic, but mainly because this is in a different category than graffiti and since my background is not in art I don\'t know what the "proper" term is , if there is one), but I do see where you are coming from. CLET tends to use the same images continuously (to a point where one could say "Is this it?") as do most street artists (I do think this term is thrown around a lot more than it should be, I agree with you there) and it can be annoying. \n tl;dr: Personal opinions \'n ****. \n'),
 ('gamingpc',
  'In Mechwarrior Online, I have begun to use a mouse for the "turret" aspect of the torso twist/tilt and a Logitech G13 for other controls 

In [16]:
spark.stop()
